In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

Wczytanie danych:

In [2]:
y_train=pd.read_csv('y_train.csv')
y_test=pd.read_csv('y_test.csv')
X_train=pd.read_csv('X_train.csv')
X_test=pd.read_csv('X_test.csv')

In [3]:
X_train

,sex,cabin,CabinReduced,cabin_map,cabin_map_reduced,sex_map
0,female,NaN,NaN,6,5,0
1,female,NaN,NaN,6,5,0
2,female,NaN,NaN,6,5,0
3,male,NaN,NaN,6,5,1
4,female,NaN,NaN,6,5,0
...,...,...,...,...,...,...
911,female,NaN,NaN,6,5,0
912,male,NaN,NaN,6,5,1
913,female,NaN,NaN,6,5,0
914,female,NaN,NaN,6,5,0


<h2> Random Forest<h2>

**Algorytm Random Forests** jest to estymator który dopasowuje wiele drzew decyzyjnych do próbek pobranych z badanego zbioru danych.
Przy pomocy uśredniania steruje dokładnością predykcji i kontroluje zjawisko over-fittingu. 

RandomForestClassifier, parametry:
- n_estimators - liczba drzew w lesie
- random state - kontroluje losowość procesu bootstrapingu 

**Dane o dużej liczebności cech:**

In [4]:
#Tworzenie modelu
RF = RandomForestClassifier(n_estimators=220,random_state=42)

#Trenowanie modelu
RF.fit(X_train[['sex_map','cabin_map']],np.ravel(y_train))

#Predykcja
PredTrain = RF.predict_proba(X_train[['sex_map','cabin_map']])
PredTest = RF.predict_proba(X_test[['sex_map','cabin_map']].fillna(0))



In [5]:
print('Dane testowe: {}'.format(roc_auc_score(y_test,PredTest[:, 1])))
print('Dane treningowe: {}'.format(roc_auc_score(y_train,PredTrain[:, 1])))

Dane testowe: 0.7627758420441345
Dane treningowe: 0.8529126140314685


**Dane o małej liczebności cech:**

In [6]:
#Trenowanie modelu
RF.fit(X_train[['sex_map','cabin_map']],np.ravel(y_train))

#Predykcja
PredTrain = RF.predict_proba(X_train[['sex_map','cabin_map_reduced']])
PredTest = RF.predict_proba(X_test[['sex_map','cabin_map_reduced']].fillna(0))

In [7]:
print('Dane testowe: {}'.format(roc_auc_score(y_test,PredTest[:, 1])))
print('Dane treningowe: {}'.format(roc_auc_score(y_train,PredTrain[:, 1])))

Dane testowe: 0.7017449256125214
Dane treningowe: 0.7334544301822994


W przypadku algorytmu Random Forests dane o dużej liczebności cech uzyskały lepsze wyniki niż dane o małej liczebności, ponieważ przez dużą kardynalnośc cech nastąpiło zjawisko overfittingu -> wynik dla danych treningowych (0,85) zauważalnie lepszy niż dla danych testowych (0,76).

<h2> Logistic Regression<h2>

**Dane o małej liczebności cech**

In [8]:
#Tworzenie modelu
LR = LogisticRegression(random_state=42)

#Trenowanie modelu
LR.fit(X_train[['sex_map','cabin_map_reduced']],np.ravel(y_train))

#Predykcja
PredTrain = LR.predict_proba(X_train[['sex_map','cabin_map_reduced']])
PredTest = LR.predict_proba(X_test[['sex_map','cabin_map_reduced']].fillna(0))

In [9]:
print('Dane testowe: {}'.format(roc_auc_score(y_test,PredTest[:, 1])))
print('Dane treningowe: {}'.format(roc_auc_score(y_train,PredTrain[:, 1])))

Dane testowe: 0.8022648083623694
Dane treningowe: 0.8026507127438499



**Dane o dużej liczebności cech**

In [10]:
#Trenowanie modelu
LR.fit(X_train[['sex_map','cabin_map']],np.ravel(y_train))

#Predykcja
PredTrain = LR.predict_proba(X_train[['sex_map','cabin_map']])
PredTest = LR.predict_proba(X_test[['sex_map','cabin_map']].fillna(0))

In [11]:
print('Dane testowe: {}'.format(roc_auc_score(y_test,PredTest[:, 1])))
print('Dane treningowe: {}'.format(roc_auc_score(y_train,PredTrain[:, 1])))

Dane testowe: 0.7899867263978761
Dane treningowe: 0.8121757682186185


W przypadku algorytmu LogisticReggresion wyniki dla obu zbiorów, o dużej i małej kardynalności, są dla siebie zbliżone, przy czym dla zbioru o małej liczebności cech różnica pomiędzy zbiorem testowym i treningowym jest minimalna i wynosi jedynie 0.0004, co wskazuje wystąpienie zjawiska underfittingu. Różnica pomiędzy zbiorami treningowym i testowym dla danych o dużej liczebności cech wynosi 0.2 co wskazuje na lepsze dopasowanie.

<h2> Gradient Boosting Classifier<h2>

**Dane o dużej liczebności cech**

In [12]:
#Tworzenie modelu
GBC = GradientBoostingClassifier(n_estimators =220, random_state=42)

#Trenowanie modelu
GBC.fit(X_train[['sex_map','cabin_map']],np.ravel(y_train))

#Predykcja
PredTrain = GBC.predict_proba(X_train[['sex_map','cabin_map']])
PredTest = GBC.predict_proba(X_test[['sex_map','cabin_map']].fillna(0))

In [13]:
print('Dane testowe: {}'.format(roc_auc_score(y_test,PredTest[:, 1])))
print('Dane treningowe: {}'.format(roc_auc_score(y_train,PredTrain[:, 1])))

Dane testowe: 0.7561805209888833
Dane treningowe: 0.8604375588082864


**Dane o małej liczebości cech**

In [14]:
#Trenowanie modelu
GBC.fit(X_train[['sex_map','cabin_map_reduced']],np.ravel(y_train))

#Predykcja
PredTrain = GBC.predict_proba(X_train[['sex_map','cabin_map_reduced']])
PredTest = GBC.predict_proba(X_test[['sex_map','cabin_map_reduced']].fillna(0))

In [15]:
print('Dane testowe: {}'.format(roc_auc_score(y_test,PredTest[:, 1])))
print('Dane treningowe: {}'.format(roc_auc_score(y_train,PredTrain[:, 1])))

Dane testowe: 0.8013245948786019
Dane treningowe: 0.816719415917359


Dla Gradient Boosting Classifier Dane o dużej liczebności cech wskazują na występowanie zjawiska overfittingu (dane treningowe - 0.86, dane testowe - 0.75). Dla danych o małej kardynalności różnica pomiędzy zbiorami wynosi 0.01.

<h2> Ada Boost Classifier<h2>

**Dane o dużej liczebności**

In [20]:
#Tworzenie modelu
ADC = AdaBoostClassifier(n_estimators =220, random_state=42)

#Trenowanie modelu
ADC.fit(X_train[['sex_map','cabin_map']],np.ravel(y_train))

#Predykcja
PredTrain = ADC.predict_proba(X_train[['sex_map','cabin_map']])
PredTest = ADC.predict_proba(X_test[['sex_map','cabin_map']].fillna(0))

In [21]:
print('Dane testowe: {}'.format(roc_auc_score(y_test,PredTest[:, 1])))
print('Dane treningowe: {}'.format(roc_auc_score(y_train,PredTrain[:, 1])))

Dane testowe: 0.7776809910956253
Dane treningowe: 0.8330800698403434


**Dane o małej liczebnośći**

In [22]:
#Trenowanie modelu
ADC.fit(X_train[['sex_map','cabin_map_reduced']],np.ravel(y_train))

#Predykcja
PredTrain = ADC.predict_proba(X_train[['sex_map','cabin_map_reduced']])
PredTest = ADC.predict_proba(X_test[['sex_map','cabin_map_reduced']].fillna(0))

In [23]:
print('Dane testowe: {}'.format(roc_auc_score(y_test,PredTest[:, 1])))
print('Dane treningowe: {}'.format(roc_auc_score(y_train,PredTrain[:, 1])))

Dane testowe: 0.7999419279907085
Dane treningowe: 0.8161759896145195


Dla Ada Boost Classifier Dane o dużej liczebności cech wskazują na występowanie zjawiska overfittingu (dane treningowe - 0.83, dane testowe - 0.77). Wynik dla danych o małej kardynalności różnica pomiędzy zbiorami wynosi 0.016.

<h1> Podsumowanie<h1>

Tendencję dla nadmiernego dopasowania wykazuja algorytmy: Random Forest, Ada Boost Classifier i Gradient Boost Classifier. Dla tych algorytmów w przypadku danych o dużej liczebności cech różnice pomiędzy wynikiem dopasowania dla zbioru testowego i zbioru treningowego wynosiła około 0.1. Dla Logistic Regression różnica ta była minimalna. 

<h1> predict vs predict_proba<h1>

<b>predict()</b>  - zwraca jako wynik dokładne dopasowanie do danej klasy <br>
<b>predict_proba()</b> - zwraca jako wynik klasy prawdopodobieństwa, dla każdej potencjalnej klasy jest obliczane prawdopodobieństwo zaliczenia do niej danego elementu